### Import Libraries

In [39]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

### Connect to sqldb

In [40]:
db_path = "data/sqldb"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

sqlite
['customer_data', 'partner_data', 'transaction_data']

CREATE TABLE customer_data (
	customer_id BIGINT, 
	customer_name TEXT, 
	city TEXT
)

/*
3 rows from customer_data table:
customer_id	customer_name	city
10001	Sankar	New York
10002	Ragul	Los Angeles
10003	Aswin	Boston
*/


CREATE TABLE partner_data (
	partner_id BIGINT, 
	partner_name TEXT, 
	partner_code TEXT
)

/*
3 rows from partner_data table:
partner_id	partner_name	partner_code
9001	American Airlines	AA
9002	Jetblue	JB
9003	GAP	GAP
*/


CREATE TABLE transaction_data (
	customer_id BIGINT, 
	partner_id BIGINT, 
	tran_amt BIGINT, 
	tran_dt DATETIME, 
	tran_status TEXT
)

/*
3 rows from transaction_data table:
customer_id	partner_id	tran_amt	tran_dt	tran_status
10001	9001	100	2024-06-15 00:00:00	success
10002	9003	150	2024-04-16 00:00:00	failed
10003	9002	90	2024-03-17 00:00:00	success
*/


### Test the access to the environment variables

In [1]:
from dotenv import load_dotenv
import os
print("Environment variables are loaded:", load_dotenv())
#print("test by reading a variable:", os.getenv("API_KEY"))

Environment variables are loaded: True


### Test your GPT model

In [4]:
import openai

openai.api_key = os.getenv("API_KEY")

messages = [
    {"role": "system", "content": str(
        "You are a helpful assistant"
    )},
    {"role": "user", "content": str("hello")}
]

# Generate the response
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    max_tokens=500
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


### Load the LLM

In [4]:
#!pip install --upgrade langchain
from langchain_openai import ChatOpenAI
api_key = os.getenv("API_KEY")

llm = ChatOpenAI(
    api_key=api_key,
    model="gpt-3.5-turbo",
    temperature=0.0
)

### 1. Chain to Generate SQL Query

In [33]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "show me the count of failed transactions for each month"})
print(response)

SELECT strftime('%m', tran_dt) AS month, COUNT(*) AS failed_transactions
FROM transaction_data
WHERE tran_status = 'failed'
GROUP BY month
ORDER BY month;


In [8]:
response = chain.invoke({"question": "show me the most preffered partners for the customers who lives in new york?"})
print(response)

SELECT partner_data.partner_name, COUNT(transaction_data.partner_id) AS total_transactions
FROM customer_data
JOIN transaction_data ON customer_data.customer_id = transaction_data.customer_id
JOIN partner_data ON transaction_data.partner_id = partner_data.partner_id
WHERE customer_data.city = 'New York'
GROUP BY partner_data.partner_name
ORDER BY total_transactions DESC
LIMIT 5;


In [2]:
#db.run(response)

In [46]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### 2. Chain to Execute SQL Query from 1st Chain

In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "show me the count of failed transactions for each month"})

"[('04', 2), ('05', 1), ('06', 1)]"

### 3. Chain to Answer the question in a user friendly manner

In [84]:
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,PromptTemplate
    
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a SQL expert. Given an input question, create a syntactically correct SQL query to run."),
        MessagesPlaceholder(variable_name = "messages"),
        ("human","{input}"),
        ("system", "**Top K:** {top_k}"),
        ("system", "**Table Information:** {table_info}")
    ]
)
write_query = create_sql_query_chain(llm, db, final_prompt)
execute_query = QuerySQLDataBaseTool(db=db)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(result=itemgetter("query") | execute_query)
)

history = ChatMessageHistory()
question = "how many customers are there from new york"
response = chain.invoke({"question": question, "messages": history.messages})
history.add_user_message(question)
history.add_ai_message(response)
print(response)

print(type(history.messages))

ValidationError: 2 validation errors for AIMessage
content.str
  Input should be a valid string [type=string_type, input_value={'question': 'how many cu...sqlalche.me/e/20/e3q8)'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
content.list[union[str,dict[any,any]]]
  Input should be a valid list [type=list_type, input_value={'question': 'how many cu...sqlalche.me/e/20/e3q8)'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/list_type

In [29]:
question = "can you display their names?"
response = chain.invoke({"question": question, "messages": history.messages})
history.add_user_message(question)
history.add_ai_message(response)

In [30]:
print(response)

print(history)

```sql
SELECT customer_name
FROM customer_data
WHERE city = 'New York';
```
Human: how many customers are there from new york
AI: ```sql
SELECT COUNT(*) AS total_customers
FROM customer_data
WHERE city = 'New York';
```
Human: can you display their names?
AI: ```sql
SELECT customer_name
FROM customer_data
WHERE city = 'New York';
```


In [69]:
def get_prompt(history):
    history = ChatMessageHistory()
    if history:  # Check if history is not empty
        for turn in history:
            if turn['role'] == 'user':
                history.add_user_message(turn['content'])
            elif turn['role'] == 'assistant':
                history.add_ai_message(turn['content'])
    return history

# Initialize history
history = ChatMessageHistory()
history.add_user_message("Hello, how are you?")
history.add_ai_message("I'm doing well, thank you. How about you?")


question = "how many customers are there from new york"
prompt = get_prompt(history)

<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>


### 4. Use Agents for the Same Task

In [50]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [51]:
agent_executor.invoke(
    {
        "input": "List the total customers per city. Which city has maximum customers?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


customer_data, partner_data, transaction_data
Invoking: `sql_db_schema` with `{'table_names': 'customer_data'}`



CREATE TABLE customer_data (
	customer_id BIGINT, 
	customer_name TEXT, 
	city TEXT
)

/*
3 rows from customer_data table:
customer_id	customer_name	city
10001	Sankar	New York
10002	Ragul	Los Angeles
10003	Aswin	Boston
*/
Invoking: `sql_db_query` with `{'query': 'SELECT city, COUNT(customer_id) AS total_customers FROM customer_data GROUP BY city ORDER BY total_customers DESC'}`


[('New York', 4), ('Los Angeles', 4), ('Seattle', 2), ('Chicago', 2), ('Boston', 2)]The total customers per city are as follows:
- New York: 4 customers
- Los Angeles: 4 customers
- Seattle: 2 customers
- Chicago: 2 customers
- Boston: 2 customers

The city with the maximum customers is both New York and Los Angeles, each with 4 customers.

> Finished chain.


{'input': 'List the total customers per city. Which city has maximum customers?',
 'output': 'The total customers per city are as follows:\n- New York: 4 customers\n- Los Angeles: 4 customers\n- Seattle: 2 customers\n- Chicago: 2 customers\n- Boston: 2 customers\n\nThe city with the maximum customers is both New York and Los Angeles, each with 4 customers.'}